In [1]:
#Import necessary libraries to fetch the train and test data...
import os
from zipfile import ZipFile
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf

from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
from keras.layers import LeakyReLU

tf.set_random_seed(42)

C:\Users\Iyyappan\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
project_path = "D:/Ganga/AIML/AIML_ML/Hackathon"

In [3]:
df_labels = pd.read_csv('D:/Ganga/AIML/AIML_ML/Hackathon/train_labels.csv')

In [4]:
df_labels.head()

,ImageName,Annotation,Class
0,./images/3917.jpg,./annotations/3917.mat,58
1,./images/5368.jpg,./annotations/5368.mat,85
2,./images/1327.jpg,./annotations/1327.mat,18
3,./images/5544.jpg,./annotations/5544.mat,88
4,./images/6145.jpg,./annotations/6145.mat,95


In [5]:
df_labels.shape

(4346, 3)

In [6]:
print('Total count of Images: ', df_labels.shape[0])
print('Number of Image Category: ', len(df_labels.Class.unique()))

Total count of Images:  4346
Number of Image Category:  97


In [7]:
print('Count of each category of Images:')
df_labels.groupby('Class').size()

Count of each category of Images:


Class
0     133
1      36
2      29
3      30
4      35
5      41
6      31
7      25
8      95
9      69
10     25
11     54
12     65
13     36
14     31
15     71
16     38
17     46
18     45
19     73
20     33
21     54
22     46
23     46
24     33
25     32
26     40
27     37
28     31
29     45
     ... 
67     37
68     35
69     22
70     44
71     60
72     43
73     34
74     31
75     41
76     23
77     54
78     45
79     24
80     52
81     34
82     62
83     40
84     48
85     26
86     64
87     32
88     65
89     59
90    164
91     26
92     44
93     24
94     39
95     30
96     43
Length: 97, dtype: int64

In [8]:
labels = tf.keras.utils.to_categorical(df_labels.Class)

In [9]:
labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [10]:
labels.shape

(4346, 97)

In [26]:
X_train = []

ImgCnt = df_labels.shape[0]

for i in range(ImgCnt):
  try:
      dummy = cv2.imread(df_labels.ImageName[i])
      dummy = cv2.resize(dummy,(150,150)) #resize to have all the images of same size
      X_train.append(dummy)
  except Exception as e:
      print(e)

In [27]:
print('x_train Count: ', len(X_train))

x_train Count:  4346


In [28]:
#Convert the list to numpy array for easy manipulation...
X_train_arr = np.asarray(X_train)

In [29]:
X_train_std = X_train_arr/255

In [31]:
X_train_std = X_train_std.reshape(X_train_std.shape[0], 150, 150, 3).astype('float32')

In [32]:
from sklearn.model_selection import train_test_split

X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_std, labels, test_size=0.2, random_state=2)
print ("No. of images in train dataset: ", len(X_train_split))
print ("No. of images in Validation dataset: ", len(X_val_split))

No. of images in train dataset:  3476
No. of images in Validation dataset:  870


In [21]:
model = Sequential()
model.add(Conv2D(filters=32, input_shape = (128,128,3), kernel_size=5)) 
model.add(Conv2D(filters=64, kernel_size=3))

model.add(Flatten()) 

# fully connected layer
model.add(Dense(units=256, kernel_initializer = 'he_normal', activation = 'relu'))
model.add(Dense(units = 97, activation = 'softmax'))

In [22]:
model.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [ ]:
history = model.fit(X_train_split, y_train_split,
                    epochs=10, 
                    validation_data=(X_val_split, y_val_split),
                    verbose = 1,
                    batch_size=128)

In [25]:
from keras.applications import InceptionResNetV2

conv_base = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(150,150,3))

219062272/219055592 [==============================] - 420s 2us/step


In [33]:
model = Sequential()
model.add(conv_base)
#model.add(Conv2D(filters=32, input_shape = (128,128,3), kernel_size=5)) 
#model.add(Conv2D(filters=64, kernel_size=3))

model.add(Flatten()) 

# fully connected layer
model.add(Dense(units=256, kernel_initializer = 'he_normal', activation = 'relu'))
model.add(Dense(units = 97, activation = 'softmax'))

In [34]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Model)  (None, 3, 3, 1536)        54336736  
_________________________________________________________________
flatten_4 (Flatten)          (None, 13824)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               3539200   
_________________________________________________________________
dense_8 (Dense)              (None, 97)                24929     
Total params: 57,900,865
Trainable params: 57,840,321
Non-trainable params: 60,544
_________________________________________________________________


In [35]:
print ("No. of trainable weights before freezing the conv_base: ", len(model.trainable_weights))
conv_base.trainable = False
print ("No. of trainable weights after freezing the conv_base: ", len(model.trainable_weights))

No. of trainable weights before freezing the conv_base:  492
No. of trainable weights after freezing the conv_base:  4


In [36]:
model.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [37]:
from keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator( rotation_range=90,
                 width_shift_range=0.1, height_shift_range=0.1,
                 horizontal_flip=True)
train_datagen.fit(X_train_split)

In [38]:
val_datagen = ImageDataGenerator( rotation_range=90,
                 width_shift_range=0.1, height_shift_range=0.1,
                 horizontal_flip=True)
val_datagen.fit(X_val_split)

In [39]:
train_generator = train_datagen.flow(X_train_split, y_train_split, batch_size=9)
val_generator = val_datagen.flow(X_val_split, y_val_split, batch_size=9)

In [60]:
model.fit_generator(train_generator,\
                    epochs=5, steps_per_epoch=3476//9, \
                    verbose=1,validation_data=val_generator, validation_steps = 870//9)

Epoch 1/5
386/386 [==============================] - 1589s 4s/step - loss: 3.0805 - acc: 0.2334 - val_loss: 5.6321 - val_acc: 0.1968
Epoch 2/5
386/386 [==============================] - 1181s 3s/step - loss: 3.0438 - acc: 0.2483 - val_loss: 5.4423 - val_acc: 0.2497
Epoch 3/5
386/386 [==============================] - 557s 1s/step - loss: 2.9774 - acc: 0.2510 - val_loss: 7.1668 - val_acc: 0.1858
Epoch 4/5
386/386 [==============================] - 532s 1s/step - loss: 2.9568 - acc: 0.2631 - val_loss: 4.4938 - val_acc: 0.2613
Epoch 5/5
386/386 [==============================] - 533s 1s/step - loss: 2.9114 - acc: 0.2706 - val_loss: 5.4968 - val_acc: 0.2509


In [41]:
df_Testlabels = pd.read_excel('D:/Ganga/AIML/AIML_ML/Hackathon/test_labels_students.xlsx')

In [42]:
X_test = []

ImgCntTest = df_Testlabels.shape[0]

for i in range(ImgCntTest):
    try:
        dummy = cv2.imread(df_Testlabels.ImageName[i])
        dummy = cv2.resize(dummy,(150,150)) #resize to have all the images of same size
        X_test.append(dummy)
    except Exception as e:
        print(e)

In [43]:
X_test_arr = np.asarray(X_test)
X_test_std = X_test_arr/255
X_test_std = X_test_std.reshape(X_test_std.shape[0], 150, 150, 3).astype('float32')

In [50]:
predictions_test = model.predict_classes(X_test_std)

In [51]:
type(predictions_test)

numpy.ndarray

In [52]:
df_Testlabels['Class'] = predictions_test.tolist()

In [57]:
df_Testlabels.head()

,ID,Class
0,4954,0
1,3777,43
2,3475,51
3,5166,15
4,2500,18


In [56]:
df_Testlabels.drop(columns='ImageName', inplace=True)

In [59]:
df_Testlabels.to_csv('./test1.csv')